In [1]:
from numpy.polynomial import Polynomial
from numpy.polynomial.polynomial import polydiv
from phe import paillier
from phe.util import getprimeover, invert, powmod
import numpy as np
import secrets

In [382]:
def L(x, n):
    return (x - 1) // n

class PublicKey:
    def __init__(self, n, g):
        self.n = n
        self.g = g

    def encrypt(self, m):
        n = self.n
        g = self.g
        r = 0
        while r == 0:
            r = secrets.randbelow(self.n)
        
        return (powmod(g, m, n**2)) * (powmod(r, n, n**2)) % n**2
    
    def add(self, ctext1, ctext2):
        return (ctext1 * ctext2) % self.n**2
    
    def mul(self, ctext, ptext):
        return powmod(ctext, ptext, self.n**2)
    
    def obfuscate(self, c):
        n = self.n
        r = 0
        while r == 0:
            r = secrets.randbelow(self.n)

        return c * (powmod(r, n, n**2)) % n**2


class PrivateKey:
    def __init__(self, lam, mu, pk):
        self.lam = lam
        self.mu = mu
        self.pk = pk

    def decrypt(self, c):
        lam = self.lam
        mu = self.mu
        n = self.pk.n

        return ((L(powmod(c, lam, n**2), n) % n) * mu) % n


def generateKeyPair(n_length):
    p = q = n = None
    n_len = 0
    while n_len != n_length:
        p = getprimeover(n_length // 2)
        q = getprimeover(n_length // 2)
        n = p * q
        n_len = n.bit_length()

    g = n + 1
    phi_n = (p - 1) * (q - 1)
    lam = phi_n
    mu = invert(phi_n, n)

    pk = PublicKey(n, g)
    sk = PrivateKey(lam, mu, pk)

    return pk, sk

In [353]:
class MyPolynomial:
    def __init__(self, coef):
        self.coef = []
        for x in coef:
            self.coef.append(int(x))

    def __call__(self, x, n):
        y = 0
        for i in range(self.n_coef()):
            y += (self.coef[i] * powmod(x, i, n)) % n
            y %= n
        return y

    def empty(n):
        return MyPolynomial(np.zeros(n))

    def fromroots(roots):
        new_poly = MyPolynomial([1])
        for x in roots:
            new_poly = new_poly.mul(MyPolynomial([-x, 1]))
        return new_poly

    def n_coef(self):
        return len(self.coef)
    
    def degree(self):
        return len(self.coef) - 1

    def reduce(self):
        last = self.n_coef()
        for i in range(len(self.coef)-1, -1, -1):
            if self.coef[i] != 0:
                break
            else:
                last -= 1
        self.coef = self.coef[:last]


    def add(self, poly):
        if self.n_coef() > poly.n_coef():
            sum = self.coef
            for i in range(poly.n_coef()):
                sum[i] += poly.coef[i]
            new_poly = MyPolynomial(sum)
        else:
            sum = poly.coef
            for i in range(self.n_coef()):
                sum[i] += self.coef[i]
            new_poly = MyPolynomial(sum)
        new_poly.reduce()
        return new_poly
    
    def add_paillier(self, poly, pk):
        if self.n_coef() > poly.n_coef():
            sum = self.coef
            for i in range(poly.n_coef()):
                sum[i] = pk.add(self.coef[i], poly.coef[i])
            new_poly = MyPolynomial(sum)
        else:
            sum = poly.coef
            for i in range(self.n_coef()):
                sum[i] = pk.add(self.coef[i], poly.coef[i])
            new_poly = MyPolynomial(sum)
        new_poly.reduce()
        return new_poly
    
    def mul(self, poly):
        new_coef = self.n_coef() + poly.n_coef() - 1
        new_poly = MyPolynomial.empty(new_coef)

        for i in range(self.n_coef()):
            for j in range(poly.n_coef()):
                new_poly.coef[i+j] += self.coef[i] * poly.coef[j]
        new_poly.reduce()
        return new_poly
    
    def mul_paillier(self, poly, pk):
        #we assume self is ciphertext and poly is plaintext
        new_coef = self.n_coef() + poly.n_coef() - 1
        new_poly = MyPolynomial.empty(new_coef)
        new_poly = MyPolynomial([pk.encrypt(x) for x in new_poly.coef])

        for i in range(self.n_coef()):
            for j in range(poly.n_coef()):
                new_poly.coef[i+j] = pk.add(new_poly.coef[i+j], pk.mul(self.coef[i], poly.coef[j]))
        new_poly.reduce()
        return new_poly
    
    def deriv(self):
        if self.n_coef() <= 1:
            return MyPolynomial([1])    #batota

        new_poly = MyPolynomial.empty(self.n_coef() - 1)
        for i in range(new_poly.n_coef()):
            new_poly.coef[i] = self.coef[i+1] * (i + 1)
        new_poly.reduce()
        return new_poly

In [356]:
poly1 = MyPolynomial([-1, 1])
poly1 = MyPolynomial([pk.encrypt(x) for x in poly1.coef])
poly2 = MyPolynomial([-2, 1])
#poly3 = poly1.mul(poly2)
poly3 = poly1.mul_paillier(poly2, pk)
poly3 = MyPolynomial([sk.decrypt(x) for x in poly3.coef])
print(poly3.coef)

[2, 2210756054, 1]


In [346]:
m1 = 0
m1 = pk.encrypt(m1)
m2 = 4255776987325955965
m = pk.add(m1, m2)
m = sk.decrypt(m)
print(m)

2


In [386]:
class Player:
    """
    Class of honest-but-curious adversaries.

    Parameters
    ----------
    S : set_like
    sk : secret key to a homomorpic cryptosystem
    """
    def __init__(self, S, sk):
        self.S = S
        self.k = len(S)
        self.sk = sk
        self.f_list = []
        self.N = sk.pk.n
    
    def polynomial(self):
        # cria o polinómio f a partir do conjunto S 
        self.f = MyPolynomial.fromroots(self.S)
        #self.f = MyPolynomial([x % self.N for x in self.f.coef])
        print("f:", self.f.coef)
    
    def encrypt(self, pk):
        # encipta o polinómio f com pk
        self.ef = MyPolynomial([pk.encrypt(x) for x in self.f.coef])
        self.f_list.append(self.ef)
        print("ef:", self.ef.coef)
    
    def append_f(self, encrypted_polynomial):
        # adiciona polinomio encriptado de outro jogador a f_list
        self.f_list.append(encrypted_polynomial)

    def receive_l(self, encrypted_polynomial):
        # recebe lambda_i-1
        self.lam_im1 = encrypted_polynomial
    
    def receive_p(self, encrypted_polynomial):
        #recebe p = lambda_n
        self.ep = encrypted_polynomial
        print("ep:", self.ep.coef)

    def phi(self):
        # calcula phi_i
        c = len(self.f_list)
        r = []
        for _ in range(c):
            r.append(MyPolynomial([secrets.randbelow(self.N) for _ in range(self.k + 1)]))

        sum = MyPolynomial([self.sk.pk.encrypt(0)])
        for j in range(c):
            prod = self.f_list[j].mul_paillier(r[j], self.sk.pk)
            print("prod:", prod.coef)
            #prod = self.f_list[j].mul(r[j])
            sum = sum.add_paillier(prod, self.sk.pk)
            #sum = sum.add(prod)
        self.phi_i = sum
        print("phi_i:", self.phi_i.coef)
        dec_phi_i = MyPolynomial([self.sk.decrypt(x) for x in self.phi_i.coef])
        for x in self.phi_i.coef:
            x = self.sk.pk.obfuscate(x) #re-randomização dos coeficientes

    def lam(self):
        # calcula lambda_i
        self.lam_i = self.lam_im1.add_paillier(self.phi_i, self.sk.pk)
        print("lam_i:", self.lam_i.coef)
        #self.lam_i = self.lam_im1.add(self.phi_i)
        for x in self.lam_i.coef:
            x = self.sk.pk.obfuscate(x) #re-randomização dos coeficientes

    
    def decrypt(self):
        self.p = MyPolynomial([self.sk.decrypt(x) for x in self.ep.coef])
        print("p:", self.p.coef)

    def multiset(self):
        # calcula o multiset de intersecção na forma [(a,b),...,(a,b)], a \in S_i
        # onde o elemento a aparece b vezes no multiset


        res = []
        for elem in self.S:
            degree = 0
            poly = self.p
            while poly(elem, self.N) == 0:
                degree += 1
                poly = poly.deriv()
            if degree > 0:
                res.append((elem, degree))
        self.intersection = res

    

Soma de polinómios encriptados: `a.ef + b.ef`

Produto de polinímio não encriptado com polinómio encriptado: `a.f * b.ef`

Derivada de polinómio encriptado: `a.ef.deriv(d)`

Avaliação de um polinómio encriptado num ponto não encriptado: `a.ef(x)`


Função da secção 5.1

In [114]:
def psi(players, c, pk):
    """
    Perform a private Set-Intersection secure against a coalition of honest-but-curious
    adversaries.

    Parameters
    ----------
    players : array_like construct of class Player elements
    c : dishonestly colluding constant
    pk: public key to a homomorpic cryptosystem
    """
    n = len(players)
    assert n >= 2, "Not enough players"
    assert c < n, "Dishonestly colluding too large"
    assert all([player.k == players[0].k for player in players]),\
        "Differently sized private sets"
    #1
    for player in players:
        #a
        player.polynomial()       
        player.encrypt(pk)
    
    #b
    for i in range(n):
        for j in range(1, c+1):
            players[i].append_f(players[(i-j) % n].ef)
        
        # Qual é o domínio?? #consoante a resposta é preciso alterar a função phi
        # DEFAULT_KEYSIZE = 3072
        # BASE = 16
        #c, d
        players[i].phi()

    #2
    players[0].lam_i = players[0].phi_i
    players[1].receive_l(players[0].lam_i)

    #3
    for i in range(1, n):
        #a é realizado implicitamente
        #b
        players[i].lam()

        #c
        players[(i+1)%len(players)].receive_l(players[i].lam_i)
    #4
    for player in players:
        player.receive_p(players[0].lam_i)

    #5
    for player in players:
        player.decrypt()
        pass
    #6
    for player in players:
        player.multiset()

In [146]:
# Generate keys
pk, sk = paillier.generate_paillier_keypair(None, 16)
pk.max_int = pk.nsquare

In [387]:
pk, sk = generateKeyPair(1024)

a = Player([1,2,3,4],sk)
b = Player([1,1,1,1],sk)

# Calculate private set
psi([a,b],1,pk)

f: [24, -50, 35, -10, 1]
ef: [6609830341876286138418168181796786802912771913890654850721406154844423579996649047273532638048999504515148880163841028570085352294775188364614797554825016004765500799635976915009041032532589815082835884339998286510473417684581746374827272118814582581700017241384163468414281137886587999828520239322952301952334311652013597893049892438645128322966900952103327916104591577747558981548608088701799935591088888352074713038015165135628314722315115742552602351661372176546721082939600135104968588845204525739547070801443735093652578979053185472684397697630706930132296252416983466860986443970994302575547123331852914734614, 7019387368613470471715785231734785334926112019267471251814336001394057928275563203351410228323011581465293932343564605745443550332647275824884505064751831798370420796182284358473700475588141042695658499426875051255893191297937230366734282499705211138876691285661113567485692926879830567384418643414219119223579450630508800801092755382273505142158706652

In [390]:
#find private set
for x in a.intersection:
    print(x)

for x in a.S:
    print(a.p(x, pk.n))


(1, 1)
0
69235081439890316758905643231489303029742332787737712141848597054187667516640858747812008484802851413977429476371791153200422231434535413660834113876753618078483877637486017174545847902705950600199700269794418662682898155243199784488187763629358697175671253154314762902485466918751717408854562356203847177459
21383759996856595007746814565947275178911213502802723356864062287203575419064356243800377656739917395852384064078909206060084006320316898509482564601462393969210101877381945242308136191691921196807253255486773623896107452954262886252796595929507118893973910085543794839354045787997151750443734989297922320400
58774748568285500062434374297735110920217462857631397388421539816972268687728947257903506862883101815614758976265263228824828701539107273269402764866335255454570430285828674494478169040815514919571412905210065163724470080913437747066227661156519848665069217487585597304104624201570525294005163136332952758048


In [177]:
m = MyPolynomial([1000, 5000, 500])
pk, sk = generateKeyPair(16)
print("key generated")
c = MyPolynomial([pk.encrypt(x) for x in m.coef])
print(c.coef)
d = MyPolynomial([sk.decrypt(x) for x in c.coef])
print(d.coef)

key generated
44462 <class 'int'> 1000 <class 'int'> 1976780521 <class 'int'> 19282 <class 'int'> 44461 <class 'int'>
44462 <class 'int'> 5000 <class 'int'> 1976780521 <class 'int'> 34159 <class 'int'> 44461 <class 'int'>
44462 <class 'int'> 500 <class 'int'> 1976780521 <class 'int'> 41930 <class 'int'> 44461 <class 'int'>
[675046127, 1958857151, 716068006]
[1000, 5000, 500]


In [135]:
#testes
a = [1, 2, 1] #experimentem assim e com [1., 2., 1.,]  o polinomio força os ints a floats e lixa a encriptação!
b = MyPolynomial(a)
print(a)
print(b.coef)
c = [sk.pk.encrypt(x) for x in a]
d = MyPolynomial([sk.pk.encrypt(x) for x in b.coef])

e = [sk.decrypt(x) for x in c]
f = MyPolynomial([sk.decrypt(x) for x in d.coef])
print(e)
print(f.coef)

[1, 2, 1]
[1 2 1]
47942 <class 'int'> 1 <class 'int'> 2298339481 <class 'int'> 40222 <class 'int'> 47941 <class 'int'>
47942 <class 'int'> 2 <class 'int'> 2298339481 <class 'int'> 3918 <class 'int'> 47941 <class 'int'>
47942 <class 'int'> 1 <class 'int'> 2298339481 <class 'int'> 4310 <class 'int'> 47941 <class 'int'>
47942 <class 'int'> 1 <class 'int'> 2298339481 <class 'int'> 22535 <class 'int'> 47941 <class 'int'>
47942 <class 'int'> 2 <class 'int'> 2298339481 <class 'int'> 27999 <class 'int'> 47941 <class 'int'>
47942 <class 'int'> 1 <class 'int'> 2298339481 <class 'int'> 44968 <class 'int'> 47941 <class 'int'>


OverflowError: Python int too large to convert to C long

In [131]:
a = secrets.randbelow(pk.n) * secrets.randbelow(pk.n) + secrets.randbelow(pk.n) * secrets.randbelow(pk.n) + secrets.randbelow(pk.n) * secrets.randbelow(pk.n)
print(a % pk.n, a)
b = pk.encrypt(a)
c = sk.decrypt(b)
print(c)

42198 2481564240
47942 <class 'int'> 2481564240 <class 'int'> 2298339481 <class 'int'> 4510 <class 'int'> 47941 <class 'int'>
42198


In [14]:
# Teste das operações definidas em 4.2
pk, sk = paillier.generate_paillier_keypair()
a = Player([1,2,3],sk)
b = Player([2,3,4],sk)
a.polynomial()
b.polynomial()
a.encrypt(pk)
b.encrypt(pk)

d = 2
x = 0
print(f"Soma:\n{a.ef + b.ef}")
print(f"Multiplicação:\n{a.f * b.ef}")
print(f"Derivada:\n{a.ef.deriv(d)}")
print(f"Avaliação:\n{a.ef(x)}")


Soma:
-30.0 + 37.0 x - 15.0 x**2 + 2.0 x**3
Multiplicação:
144.0 - 420.0 x + 484.0 x**2 - 285.0 x**3 + 91.0 x**4 - 15.0 x**5 +
1.0 x**6
Derivada:
-12.0 + 6.0 x
Avaliação:
-6.0
